In [7]:
import numpy as np
import pandas as pd
from PIL import Image

from sklearn.cluster import DBSCAN

import os
from os import path
import tqdm

In [2]:
# Set the number of ions on images
N_ions = 4

In [5]:
def process_image(imgpath):
    img = Image.open(imgpath)
    img.load()
    img2 = np.mean(np.asarray(img, dtype="int32"), axis=2)
    # Threshold the image
    x, y = np.where(img2 > 1.1 * np.median(img2))
    
    # Find atoms as clusters. Detect clusters by their density.
    db = DBSCAN(eps=2, min_samples=10).fit(np.column_stack(((x, y))))
    
    data = np.zeros(N_ions, dtype="int32")

    labels = db.labels_
    for label in set(labels):
        if label == -1:
            continue
        
        # determine position of this cluster and determine a number o ion
        xi = x[labels==label]
        center_x = np.mean(xi)
        n_this = (center_x - np.min(x)) // ((np.max(x) - np.min(x)) // N_ions)
        data[int(n_this)] = 1
    return data

In [6]:
filenames = []
qubit_states = []

work_dir = path.join(os.getcwd(), 'Входные данные')
for f in tqdm.tqdm(os.listdir(work_dir)):
    if not f.endswith('.png'):
        continue
    full_path = path.join(work_dir, f)
    if path.isdir(full_path): continue
    states = process_image(full_path)
    filenames.append(f)
    qubit_states.append(states)
    
qubit_states = np.array(qubit_states)
save_dict = {'file number': list(range(1, len(filenames) + 1)), 'file name': filenames}
for i in range(N_ions):
    save_dict[f'qubit {i + 1} state'] = qubit_states[:, i]
df = pd.DataFrame(save_dict)
df.to_csv('labeled_ions_team_number.csv', header=True, index=False)